In [94]:
import pandas as pd
import numpy as np
import time

df = pd.read_csv('puzzle_batches/small_batch.csv')
df = df[['FEN','Moves','Themes']].dropna()

In [95]:
# Execute computer move

import chess

for row in df.itertuples():
    board = chess.Board(row.FEN)

    computer_move = row.Moves.split()[0]
    board.push_uci(computer_move)
    df.loc[row.Index].FEN = board.fen()

df.drop(['Moves'], inplace=True, axis=1)

In [96]:
# Create valid_themes columns

start = time.time()

used_themes = {
    "advancedPawn", "advantage", "bishopEndgame", "capturingDefender", "crushing", "equality", "kingsideAttack", "defensiveMove", "endgame",
    "exposedKing", "fork", "hangingPiece", "interference", "knightEndgame", "middlegame", "opening", "pawnEndgame", "pin", "promotion", "queenEndgame",
    "queenRookEndgame", "queensideAttack", "rookEndgame", "skewer", "trappedPiece", "underPromotion", "xRayAttack", "zugzwang",
}

df['valid_themes'] = ""
for row in df.itertuples():
    row_themes = set(row.Themes.split())
    valid_themes = row_themes.intersection(used_themes)
    #print(f"valid themes {valid_themes}")

    df.loc[row.Index].valid_themes = " ".join(valid_themes)

themes_dummies = df.valid_themes.str.get_dummies(' ').add_prefix('theme_')
df = pd.concat([df, themes_dummies], axis=1)

df.drop(['Themes', 'valid_themes'], inplace=True, axis=1)
end = time.time()
print(end - start)

0.005467653274536133


In [97]:
# Transform FEN into board positions and pieces

start = time.time()

for row in df.itertuples():
    board = chess.Board(row.FEN)

    # color -> false = preto / true = branco
    for pos, piece in board.piece_map().items():
        square_name = chess.square_name(pos)
        piece_name = piece.symbol().lower()

        if piece_name == "q":
            df.at[row.Index, f"{square_name}.r"] = 1 if piece.color else -1
            df.at[row.Index, f"{square_name}.b"] = 1 if piece.color else -1
        else:
            df.at[row.Index, f"{square_name}.{piece_name}"] = 1 if piece.color else -1


end = time.time()
print(end - start)

0.05980491638183594
